In [1]:
from tile_bank_repository import TileBankRepository
from datetime import datetime
import numpy as np

## Initialize TileBank Repository

In [2]:
repo = TileBankRepository()

In [3]:
repo.get_table_schema("satellite")

,column_name,column_type,null,key,default,extra
0,id,INTEGER,NO,PRI,nextval('seq_satellite_id'),None
1,name,VARCHAR,NO,None,None,None
2,resolution_cm,INTEGER,NO,None,None,None
3,type,"ENUM('optic', 'radar')",NO,None,None,None
4,date_created,TIMESTAMP,NO,None,CURRENT_TIMESTAMP,None


In [4]:
repo.get_all("satellite", ["name", "type"])

#repo.get_all("satellite2") # should raise an error since the table does not exist

,name,type
0,Sentinel-2,optic
1,Sentinel-1,radar
2,Pleiades-50,optic
3,PleiadesNEO,optic
4,ortophoto25,optic


In [5]:
repo.find("satellite", name="Sentinel-2")

,id,name,resolution_cm,type,date_created
0,1,Sentinel-2,100,optic,2025-07-11 17:36:02.513


In [6]:
repo.get_by_id("satellite", 4)

#repo.get_by_id("satellite", 56) # should raise an error since the record does not exist

,id,name,resolution_cm,type,date_created
0,4,PleiadesNEO,30,optic,2025-07-11 17:36:02.524


## Adding new records

We can add new records to the db that are saved in files already like so:

In [7]:
repo.add_single_tile_from_path("0_4736.npy", "Sentinel-2", datetime.now())

# second attempt to add the same tile should raise an error
# repo.add_single_tile_from_path("0_47362.npy", "Sentinel-2", datetime.now())

# providing an invalid path should raise an error
#repo.add_single_tile_from_path("invalid_path.npy", "Sentinel-2", datetime.now())

# providing an invalid satellite name should raise an error
#repo.add_single_tile_from_path("0_4736.npy", "Invalid-Satellite", datetime.now())

# providing an invalid date should raise an error
#repo.add_tile("0_4736.npy", "Sentinel-2", "invalid_date")


ValueError: File 0_4736.npy does not exist

In [8]:
repo.get_by_id("tile", 1)

,id,path,satellite_id,date_created,date_origin
0,1,0_4736.npy,1,2025-07-11 17:41:57.399,2025-07-11


We can also give a numpy array and have the repository save it dynamically

In [9]:
dummy_array = np.random.rand(100, 100)

repo.add_single_tile_from_array(dummy_array, "Sentinel-2", datetime.now())

,id,path,satellite_id,date_created,date_origin
0,2,.\ea44132f-53f7-4215-8fa0-497cafb1d4fa.npy,1,2025-07-11 17:41:57.399,2025-07-11


In [ ]:
repo.add_timeseries_from_path(
    paths=["timestep_1.npy", "timestep_2.npy"],
    satellite_name="Sentinel-2",
    date_origins=[datetime.now(), datetime.now()]
)

,id,start_date,end_date,date_created
0,1,2025-07-11,2025-07-11,2025-07-11 17:41:57.399


In [10]:
repo.save_changes()

In [ ]:
from datetime import timedelta


dummy_high_res_data = np.random.rand(3, 100, 100)
dummy_timeseries_data = np.random.rand(2, 3, 100, 100)

repo.add_multimodal_from_array(dummy_high_res_data, dummy_timeseries_data, "Sentinel-2", datetime.now(), [datetime.now() + timedelta(days=1), datetime.now() + timedelta(days=2)])

2025-07-15
   id                                        path  satellite_id  \
0   1  .\c5fe1363-40b0-41a0-8724-6b65643c6e4c.npy             1   

             date_created date_origin  
0 2025-07-15 15:00:58.742  2025-07-15  
2025-07-16
   id                                        path  satellite_id  \
0   2  .\9387c7fe-55a8-4033-a2b2-50a487733c03.npy             1   

             date_created date_origin  
0 2025-07-15 15:00:58.742  2025-07-16  
2025-07-17
   id                                        path  satellite_id  \
0   3  .\f939ec44-9aad-4f41-a711-4a3f5ebe7dd6.npy             1   

             date_created date_origin  
0 2025-07-15 15:00:58.742  2025-07-17  


,id,timeseries_id,high_resolution_id,date_created
0,1,1,1,2025-07-15 15:00:58.742
